<a href="https://colab.research.google.com/github/ankit-rathi/Tradevesting/blob/main/myProspects_Upside_Analysis_ATH.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import datetime
import pytz
def get_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

get_time()
gen_output = 0

Run date time (IST): 2023-11-07 15:48:06


In [2]:
# install libraries
!pip install yfinance
!pip install pandas-ta
#!pip install talib-binary

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.1/115.1 kB 2.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pandas-ta: filename=pandas_ta-0.3.14b0-py3-none-any.whl size=218908 sha256=2253eb9856786aba37b32a14b596dbf9e72d77080d5d6e3f471a0d8cb495da52
  Stored in directory: /root/.cache/pip/wheels/69/00/ac/f7fa862c34b0e2ef320175100c233377b4c558944f12474cf0
Successfully built pandas-ta


In [3]:
# set start and end date
import datetime
start_date = (datetime.date.today() + datetime.timedelta(days=-365*5)).strftime('%Y-%m-%d')
start1_date = (datetime.date.today() + datetime.timedelta(days=-1*365)).strftime('%Y-%m-%d')
end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')

In [4]:
# fetch nifty200 scrips

import pandas as pd
import numpy as np

def read_file(path):
 df = pd.read_csv(path)
 return df

path_mypf = 'https://raw.githubusercontent.com/ankit-rathi/Tradevesting/main/data/'

mypfs = pd.read_csv(path_mypf + 'myPortfolioStocks.csv')
mypps = pd.read_csv(path_mypf + 'myProspectsScrips.csv')

df_mypf = pd.merge(mypfs, mypps, on="Symbol")


def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'ADANITRANS', 'SREERAYALK', 'SCHAEFFLER ']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

In [5]:
df_mypf = df_mypf[df_mypf['InPortfolio'].isnull()]
cols = ['Symbol', 'Criteria', 'Strategy', 'LatestQtr', 'Target','Breakout','StarStock', 'BizCheck', 'FinCheck', 'TechCheck', 'Conviction', 'Category', 'Stock', 'MedianPE']
df_mypf = df_mypf[cols]
all_prospects = get_stock_ids(df_mypf)

In [6]:
def print_distribution():

  # star stocks
  print('Star Prospects:')
  print(df_mypf['StarStock'].value_counts())
  print('-------')

  # v40/v40n stocks
  print('V40/V40N Prospects:')
  print(df_mypf['Criteria'].value_counts())
  print('-------')

  # latest quarter
  print('Latest Qtr:')
  print(df_mypf['LatestQtr'].value_counts())
  print('-------')

  # star + v40/v40n stocks
  print('Star + V40/V40N Stocks:')
  print(df_mypf[(df_mypf['Criteria'] == 'V40') | (df_mypf['Criteria'] == 'V40N')]['StarStock'].value_counts())
  print('-------')

print_distribution()

Star Prospects:
0    69
1    62
Name: StarStock, dtype: int64
-------
V40/V40N Prospects:
V40N    28
V40     21
VR       6
AR       4
V200     1
Name: Criteria, dtype: int64
-------
Latest Qtr:
1    121
0     10
Name: LatestQtr, dtype: int64
-------
Star + V40/V40N Stocks:
1    25
0    24
Name: StarStock, dtype: int64
-------


In [7]:
# import libraries

%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime
from tabulate import tabulate
import warnings
warnings.filterwarnings('ignore')
import pandas_datareader.data as web
import yfinance as yfin
#import talib
yfin.pdr_override()
import pandas_ta as ta

from google.colab import data_table

In [8]:
# fetch current PE
import requests
from bs4 import BeautifulSoup

def get_current_features(stock_name):

  link_blueprint = "https://www.screener.in/company/"

  stocks = ['INDIGOPNTS', 'IRCTC', 'GILLETTE', 'PAGEIND', 'BANDHANBNK', 'ABBOTINDIA', 'JCHAC',
              'NESTLEIND', 'PGHH', 'EQUITASBNK', 'SBICARD', 'ASTRAZEN', 'ICICIGI', 'RELAXO']

  if stock_name in stocks:
    actual_link = link_blueprint + stock_name.upper()
  else:
    actual_link = link_blueprint + stock_name.upper() + "/consolidated/"

  page = requests.get(actual_link)
  #print(actual_link)
  link_soup = BeautifulSoup(page.content, 'html.parser')

  test_list = link_soup.find(id="top").text.replace("  ", "").split('\n')

  while("" in test_list) :
      test_list.remove("")
  while("\r" in test_list) :
      test_list.remove("\r")
  while(" \r" in test_list) :
      test_list.remove(" \r")

  try:

    current_pe = float(test_list[test_list.index('Stock P/E')+1:test_list.index('Stock P/E')+2][0])
    current_roce = float(test_list[test_list.index('ROCE')+1:test_list.index('ROCE')+2][0])
    current_roe = float(test_list[test_list.index('ROE')+1:test_list.index('ROE')+2][0])
  except ValueError as ve:
    current_pe = 1000
    current_roce = 1000
    current_roe = 1000

  return current_pe, current_roce, current_roe

In [9]:
# function to compute stock attributes
def get_common_features():
  def stock_prec_dev(stock_symbol):
      #stock_symbol = 'ULTRACEMCO.NS'
      short_window = 20
      mid_window = 50
      long_window = 200
      moving_avg = 'SMA'

      # import the closing price data of the stock for the aforementioned period of time in Pandas dataframe
      start = datetime.datetime(*map(int, start_date.split('-')))
      end = datetime.datetime(*map(int, end_date.split('-')))
      stock_df = yfin.Ticker(stock_symbol).history(period='5y', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]
      stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
      stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = long_window, min_periods = 1).mean(),0)
      stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
      stock_df.dropna(axis = 0, inplace = True) # remove any null rows

      stock_df['Stock'] = stock_symbol

      stock_df['Close'] = round(stock_df['Close'],2)
      stock_df['Max'] = round(max(stock_df['Close']),2)
      stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start1_date)]
      stock_df['Min'] = round(min(stock_df['Close']),2)
      stock_df['Gained%Min'] = round((stock_df['Close'] - stock_df['Min'])*100/stock_df['Min'],2)
      stock_df.ta.rsi(append=True)
      #stock_df.rename(columns={'RSI_14':'RSI'}, inplace=True)
      stock_df['Close'] = round(stock_df['Close'],0)
      stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
      stock_df.drop(['Open', 'Low', 'High', 'Volume'
                    ], axis=1, inplace=True)

      return stock_df

  df_prec_dev = pd.DataFrame()

  #tmp = stock_prec_dev('ULTRACEMCO.NS')
  #df_perform = df_perform.append(tmp)
  for stock_id in all_prospects:
      #print(stock_id)
      current_pe, current_roce, current_roe = get_current_features(stock_id)
      stock_id = stock_id.upper() + '.NS'
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      tmp = tmp.tail(1)
      tmp['Curr_PE'] = current_pe
      tmp['Curr_ROCE%'] = current_roce
      tmp['Curr_ROE%'] = current_roe
      df_prec_dev = df_prec_dev.append(tmp)
  return df_prec_dev, stock_prec_dev

df_common_features, stock_prec_dev = get_common_features()

In [10]:
# nifty in last 5 weeks
stock_prec_dev('^NSEI')[-21::5]

,Close,200_SMA,Dev%_200,Stock,Max,Min,Gained%Min,RSI_14
Date,,,,,,,,
2023-10-09 00:00:00+05:30,19512.0,18524.0,5.34,^NSEI,20192.35,16945.05,15.15,45.0
2023-10-16 00:00:00+05:30,19732.0,18563.0,6.30,^NSEI,20192.35,16945.05,16.45,52.0
2023-10-23 00:00:00+05:30,19282.0,18600.0,3.67,^NSEI,20192.35,16945.05,13.79,36.0
2023-10-31 00:00:00+05:30,19080.0,18625.0,2.44,^NSEI,20192.35,16945.05,12.60,37.0
2023-11-07 00:00:00+05:30,19407.0,18657.0,4.02,^NSEI,20192.35,16945.05,14.53,50.0


In [11]:
stock_ids = df_common_features['Stock'].values
df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)

In [12]:
df_prospects = pd.merge(df_common_features, df_mypf, on='Stock')

In [13]:
#df_common_features['Discount%PE'] = round((df_common_features['MedianPE']-df_common_features['Curr_PE'])*100/(df_common_features['MedianPE']),0)
df_prospects['Dev%_PE'] = round((df_prospects['Curr_PE'] - df_prospects['MedianPE'])*100/df_prospects['MedianPE'],2)
#df_prospects['Target'] = np.where(df_prospects['Strategy']== 'BTT', df_prospects["Target"], df_prospects['Max'])
df_prospects['Upside%Target'] = round((df_prospects['Max']-df_prospects['Close'])*100/(df_prospects['Close']),0)
df_prospects.drop([ '200_SMA', 'Symbol', 'Target', 'Max', 'Min', 'Close', 'BizCheck',	'FinCheck',	'TechCheck', 'MedianPE', 'Curr_PE'], axis=1, inplace=True)
df_prospects = df_prospects.set_index('Stock')

In [14]:
# v40 and v40n
df_tmp = df_prospects[df_prospects['Criteria'].isin(['V40','V40N'])]
data_table.DataTable(df_tmp, include_index=True)

,Dev%_200,Gained%Min,RSI_14,Curr_ROCE%,Curr_ROE%,Criteria,Strategy,LatestQtr,Breakout,StarStock,Conviction,Category,Dev%_PE,Upside%Target
Stock,,,,,,,,,,,,,,
3MINDIA.NS,18.86,46.16,59.0,31.40,23.40,V40N,ATH,1,NaN,1,M,MISC,-2.48,5.0
ABBOTINDIA.NS,7.23,24.51,67.0,41.10,31.60,V40,BTT,1,NaN,1,H,PHARMA,NaN,3.0
AKZOINDIA.NS,0.40,18.96,43.0,34.70,26.10,V40,BTT,1,NaN,1,H,NaN,NaN,19.0
ANGELONE.NS,67.21,158.70,63.0,44.00,47.10,V40N,ATH,1,NaN,1,M,FINANCE,3.98,11.0
ASTRAZEN.NS,26.94,51.88,60.0,30.90,23.00,V40N,ATH,1,NaN,1,M,PHARMA,-29.40,0.0
AXISBANK.NS,9.66,24.01,59.0,6.15,14.50,V40,ATH,1,NaN,1,H,BANKS,-56.87,2.0
BAJAJ-AUTO.NS,24.15,62.89,64.0,26.20,20.20,V40,ATH,1,NaN,1,H,AUTO,17.89,1.0
BAJAJHLDNG.NS,6.43,28.53,56.0,9.71,11.20,V40,ATH,1,NaN,0,H,FINANCE,10.94,5.0
BAJFINANCE.NS,9.99,36.60,42.0,11.80,23.50,V40,BTT,1,NaN,1,H,FINANCE,-34.22,9.0


In [15]:
# all other prospects
df_tmp = df_prospects[~df_prospects['Criteria'].isin(['V40','V40N'])]
data_table.DataTable(df_tmp, include_index=True)

,Dev%_200,Gained%Min,RSI_14,Curr_ROCE%,Curr_ROE%,Criteria,Strategy,LatestQtr,Breakout,StarStock,Conviction,Category,Dev%_PE,Upside%Target
Stock,,,,,,,,,,,,,,
ACC.NS,-0.56,16.46,36.0,10.00,7.06,NaN,NaN,0,NaN,0,L,CEMENT,NaN,46.0
ADANIENT.NS,-1.00,87.22,36.0,9.49,9.63,NaN,NaN,1,NaN,0,L,NaN,NaN,86.0
ADANIGREEN.NS,-3.10,101.68,51.0,7.81,22.80,NaN,NaN,1,NaN,1,L,NaN,NaN,219.0
ADANIPORTS.NS,10.62,73.62,50.0,9.53,14.30,NaN,NaN,1,NaN,1,L,NaN,NaN,21.0
ALKYLAMINE.NS,-10.63,2.00,41.0,41.30,41.40,NaN,NaN,0,NaN,0,L,CHEMICALS,NaN,110.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
VGUARD.NS,7.24,21.50,41.0,15.20,12.60,NaN,NaN,1,NaN,0,L,NaN,NaN,8.0
VOLTAS.NS,-1.17,10.35,36.0,9.67,4.42,NaN,NaN,1,NaN,0,L,AC,103.23,62.0
WSTCSTPAPR.NS,30.67,59.74,53.0,52.90,44.80,VR,ATH,0,NaN,0,L,PAPER,NaN,5.0


In [16]:
# top 10 prospects to invest from BTT stocks
df_prospects[df_prospects['Strategy'] == 'BTT'].sort_values(by = 'Upside%Target', ascending=False).head(10)

,Dev%_200,Gained%Min,RSI_14,Curr_ROCE%,Curr_ROE%,Criteria,Strategy,LatestQtr,Breakout,StarStock,Conviction,Category,Dev%_PE,Upside%Target
Stock,,,,,,,,,,,,,,
BLUESTARCO.NS,26.02,70.02,64.0,24.40,23.40,AR,BTT,1,NaN,1,L,AC,22.49,57.0
UPL.NS,-16.24,3.75,36.0,14.30,13.40,AR,BTT,1,NaN,0,L,NaN,NaN,48.0
PNB.NS,31.29,84.29,59.0,4.10,3.34,VR,BTT,1,NaN,1,L,BANKS,-15.06,25.0
AKZOINDIA.NS,0.40,18.96,43.0,34.70,26.10,V40,BTT,1,NaN,1,H,NaN,NaN,19.0
BAJFINANCE.NS,9.99,36.60,42.0,11.80,23.50,V40,BTT,1,NaN,1,H,FINANCE,-34.22,9.0
J&KBANK.NS,57.75,181.99,56.0,4.92,13.10,VR,BTT,1,NaN,1,L,BANKS,55.21,6.0
MOTILALOFS.NS,38.91,87.21,60.0,12.90,15.60,V40N,BTT,1,NaN,1,M,FINANCE,-46.70,6.0
NMDC.NS,37.02,63.20,65.0,30.20,23.00,VR,BTT,1,NaN,0,L,MINING,57.21,6.0
UJJIVANSFB.NS,52.76,154.94,62.0,1000.00,1000.00,V40N,BTT,1,NaN,1,M,BANKS,5852.38,4.0


In [17]:
# prospects to invest from star + V40/V40N/VR + below 200 DMA
df_prospects[(df_prospects['Criteria'].notnull()) & (df_prospects['StarStock'] == 1) & (df_prospects['Dev%_200'] < 5)].sort_values(by = 'Upside%Target', ascending=False).head(10)

,Dev%_200,Gained%Min,RSI_14,Curr_ROCE%,Curr_ROE%,Criteria,Strategy,LatestQtr,Breakout,StarStock,Conviction,Category,Dev%_PE,Upside%Target
Stock,,,,,,,,,,,,,,
KANSAINER.NS,3.08,22.44,28.0,14.50,10.7,V40N,ATH,1,NaN,1,M,PAINTS,-27.55,44.0
VINATIORGA.NS,-4.53,3.49,45.0,30.40,22.6,V40N,ATH,1,NaN,1,M,CHEMICALS,-3.04,32.0
AKZOINDIA.NS,0.40,18.96,43.0,34.70,26.1,V40,BTT,1,NaN,1,H,NaN,NaN,19.0
ITC.NS,2.54,35.36,42.0,39.00,29.1,V40,ATH,1,NaN,1,H,FMCG,36.04,14.0
MARICO.NS,0.09,12.66,36.0,41.90,35.8,V40,ATH,1,NaN,1,H,FMCG,-8.54,12.0
ICICIBANK.NS,3.50,16.97,56.0,6.32,17.2,V40,ATH,1,NaN,1,H,BANKS,-31.82,5.0


In [18]:
# top 10 prospects to invest from star stocks
df_prospects[df_prospects['StarStock'] == 1].sort_values(by = 'Upside%Target', ascending=False).head(10)

,Dev%_200,Gained%Min,RSI_14,Curr_ROCE%,Curr_ROE%,Criteria,Strategy,LatestQtr,Breakout,StarStock,Conviction,Category,Dev%_PE,Upside%Target
Stock,,,,,,,,,,,,,,
ADANIGREEN.NS,-3.10,101.68,51.0,7.81,22.80,NaN,NaN,1,NaN,1,L,NaN,NaN,219.0
BLUESTARCO.NS,26.02,70.02,64.0,24.40,23.40,AR,BTT,1,NaN,1,L,AC,22.49,57.0
KANSAINER.NS,3.08,22.44,28.0,14.50,10.70,V40N,ATH,1,NaN,1,M,PAINTS,-27.55,44.0
IGL.NS,-11.59,5.08,39.0,26.60,20.50,NaN,NaN,1,NaN,1,L,NaN,NaN,41.0
VINATIORGA.NS,-4.53,3.49,45.0,30.40,22.60,V40N,ATH,1,NaN,1,M,CHEMICALS,-3.04,32.0
TASTYBITE.NS,28.99,101.30,47.0,1000.00,1000.00,NaN,NaN,1,NaN,1,L,NaN,NaN,26.0
PNB.NS,31.29,84.29,59.0,4.10,3.34,VR,BTT,1,NaN,1,L,BANKS,-15.06,25.0
MUTHOOTFIN.NS,16.81,49.05,65.0,12.10,17.90,NaN,NaN,1,NaN,1,L,FINANCE,12.00,24.0
ADANIPORTS.NS,10.62,73.62,50.0,9.53,14.30,NaN,NaN,1,NaN,1,L,NaN,NaN,21.0


In [19]:
# top 10 prospects to invest from  V40/V40N/VR
df_prospects[df_prospects['Criteria'].notnull()].sort_values(by = 'Upside%Target', ascending=False).head(10)

,Dev%_200,Gained%Min,RSI_14,Curr_ROCE%,Curr_ROE%,Criteria,Strategy,LatestQtr,Breakout,StarStock,Conviction,Category,Dev%_PE,Upside%Target
Stock,,,,,,,,,,,,,,
JCHAC.NS,-3.70,15.15,36.0,1000.0,1000.0,V40N,ATH,1,NaN,0,M,AC,1823.08,197.0
SFL.NS,-4.50,12.82,44.0,13.6,12.5,V40N,ATH,1,NaN,0,M,MISC,18.05,84.0
SYMPHONY.NS,-7.30,4.17,55.0,14.8,13.5,V40N,ATH,1,NaN,0,M,DURABLES,-6.43,69.0
DREAMFOLKS.NS,0.12,45.13,54.0,76.8,60.2,AR,ATH,0,NaN,0,L,MISC,NaN,59.0
BLUESTARCO.NS,26.02,70.02,64.0,24.4,23.4,AR,BTT,1,NaN,1,L,AC,22.49,57.0
LALPATHLAB.NS,24.53,50.35,69.0,18.2,15.0,V40N,ATH,1,NaN,0,M,HEALTHCARE,7.62,56.0
PFIZER.NS,1.26,12.97,44.0,27.2,19.8,V40,ATH,1,NaN,0,H,PHARMA,217.81,55.0
UPL.NS,-16.24,3.75,36.0,14.3,13.4,AR,BTT,1,NaN,0,L,NaN,NaN,48.0
TTKPRESTIG.NS,5.46,21.65,51.0,18.1,13.8,V40N,ATH,1,NaN,0,M,DURABLES,13.56,46.0


In [20]:
# top 10 prospects to invest below 200 DMA
df_prospects[df_prospects['Dev%_200'] < 0].sort_values(by = 'Upside%Target', ascending=False).head(10)

,Dev%_200,Gained%Min,RSI_14,Curr_ROCE%,Curr_ROE%,Criteria,Strategy,LatestQtr,Breakout,StarStock,Conviction,Category,Dev%_PE,Upside%Target
Stock,,,,,,,,,,,,,,
YESBANK.NS,-1.18,13.90,52.0,4.94,1.99,NaN,NaN,1,NaN,0,L,NaN,NaN,1527.0
ADANIGREEN.NS,-3.10,101.68,51.0,7.81,22.80,NaN,NaN,1,NaN,1,L,NaN,NaN,219.0
JCHAC.NS,-3.70,15.15,36.0,1000.00,1000.00,V40N,ATH,1,NaN,0,M,AC,1823.08,197.0
BIOCON.NS,-6.88,14.07,31.0,5.92,4.78,NaN,NaN,1,NaN,0,L,NaN,NaN,113.0
ALKYLAMINE.NS,-10.63,2.00,41.0,41.30,41.40,NaN,NaN,0,NaN,0,L,CHEMICALS,NaN,110.0
ADANIENT.NS,-1.00,87.22,36.0,9.49,9.63,NaN,NaN,1,NaN,0,L,NaN,NaN,86.0
SFL.NS,-4.50,12.82,44.0,13.60,12.50,V40N,ATH,1,NaN,0,M,MISC,18.05,84.0
SYMPHONY.NS,-7.30,4.17,55.0,14.80,13.50,V40N,ATH,1,NaN,0,M,DURABLES,-6.43,69.0
VOLTAS.NS,-1.17,10.35,36.0,9.67,4.42,NaN,NaN,1,NaN,0,L,AC,103.23,62.0


In [21]:
# star + v40/v40n prospects (for breakout)

tmp_df = df_prospects[df_prospects['StarStock'] == 1]
tmp_df = tmp_df[(tmp_df['Criteria'] == 'V40') | (tmp_df['Criteria'] == 'V40N')]
tmp_df = tmp_df[(tmp_df['Upside%Target'] < 5)]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Upside%Target', ascending=True)

qualified stocks: 11


,Dev%_200,Gained%Min,RSI_14,Curr_ROCE%,Curr_ROE%,Criteria,Strategy,LatestQtr,Breakout,StarStock,Conviction,Category,Dev%_PE,Upside%Target
Stock,,,,,,,,,,,,,,
ASTRAZEN.NS,26.94,51.88,60.0,30.90,23.0,V40N,ATH,1,NaN,1,M,PHARMA,-29.40,0.0
COLPAL.NS,23.93,50.21,61.0,179.00,158.0,V40,BTT,1,NaN,1,H,FMCG,218.18,-0.0
MCDOWELL-N.NS,21.11,47.18,65.0,20.00,17.5,V40N,ATH,1,NaN,1,M,BREWERIES,-14.36,0.0
BAJAJ-AUTO.NS,24.15,62.89,64.0,26.20,20.2,V40,ATH,1,NaN,1,H,AUTO,17.89,1.0
HCLTECH.NS,12.94,29.57,58.0,28.30,23.0,V40,ATH,1,NaN,1,H,IT,37.42,1.0
AXISBANK.NS,9.66,24.01,59.0,6.15,14.5,V40,ATH,1,NaN,1,H,BANKS,-56.87,2.0
TITAN.NS,15.47,42.14,57.0,25.10,30.8,V40,ATH,1,NaN,1,H,JEWELLERY,12.82,2.0
ABBOTINDIA.NS,7.23,24.51,67.0,41.10,31.6,V40,BTT,1,NaN,1,H,PHARMA,NaN,3.0
CAPLIPOINT.NS,31.05,85.11,56.0,26.30,22.3,V40N,ATH,1,NaN,1,M,PHARMA,18.18,3.0


In [22]:
# prospects for interactive analysis

data_table.DataTable(df_prospects, include_index=True)

,Dev%_200,Gained%Min,RSI_14,Curr_ROCE%,Curr_ROE%,Criteria,Strategy,LatestQtr,Breakout,StarStock,Conviction,Category,Dev%_PE,Upside%Target
Stock,,,,,,,,,,,,,,
3MINDIA.NS,18.86,46.16,59.0,31.40,23.40,V40N,ATH,1,NaN,1,M,MISC,-2.48,5.0
ABBOTINDIA.NS,7.23,24.51,67.0,41.10,31.60,V40,BTT,1,NaN,1,H,PHARMA,NaN,3.0
ACC.NS,-0.56,16.46,36.0,10.00,7.06,NaN,NaN,0,NaN,0,L,CEMENT,NaN,46.0
ADANIENT.NS,-1.00,87.22,36.0,9.49,9.63,NaN,NaN,1,NaN,0,L,NaN,NaN,86.0
ADANIGREEN.NS,-3.10,101.68,51.0,7.81,22.80,NaN,NaN,1,NaN,1,L,NaN,NaN,219.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
VIPIND.NS,-2.88,11.20,42.0,27.80,29.00,V40N,ATH,1,NaN,0,M,MISC,25.54,21.0
VOLTAS.NS,-1.17,10.35,36.0,9.67,4.42,NaN,NaN,1,NaN,0,L,AC,103.23,62.0
WSTCSTPAPR.NS,30.67,59.74,53.0,52.90,44.80,VR,ATH,0,NaN,0,L,PAPER,NaN,5.0


In [23]:
# save the output
if gen_output == 1:
  from google.colab import drive
  drive.mount('/content/drive')
  df_prec_dev = df_prec_dev.reset_index()
  df_prec_dev.to_csv('/content/drive/My Drive/data/stocks/myProspects-Upside-Analysis-ATH.csv', index=False)

In [24]:
# stock in last 5 weeks
#stock_ids = ['ICICIGI','DABUR','MOTILALOFS','JPPOWER','EQUITAS','GLAND','JMFINANCIL','LUXIND','UJJIVANSFB','VAIBHAVGBL','VALIANTORG']
stock_prec_dev('BERGEPAINT.NS')[-21::5]

,Close,200_SMA,Dev%_200,Stock,Max,Min,Gained%Min,RSI_14
Date,,,,,,,,
2023-10-09 00:00:00+05:30,563.0,526.0,7.00,BERGEPAINT.NS,710.11,447.11,25.88,40.0
2023-10-16 00:00:00+05:30,569.0,527.0,8.01,BERGEPAINT.NS,710.11,447.11,27.31,44.0
2023-10-23 00:00:00+05:30,570.0,530.0,7.64,BERGEPAINT.NS,710.11,447.11,27.60,45.0
2023-10-31 00:00:00+05:30,559.0,532.0,5.00,BERGEPAINT.NS,710.11,447.11,24.94,45.0
2023-11-07 00:00:00+05:30,571.0,534.0,7.01,BERGEPAINT.NS,710.11,447.11,27.81,52.0
